In [1]:
import gensim.downloader as api

In [2]:
text8_path = api.load('text8', return_path=True)
print("Using corpus from", text8_path)

Using corpus from /Users/imayak/gensim-data/text8/text8.gz


In [15]:
from gensim.models import Word2Vec, KeyedVectors
from gensim.models.word2vec import Text8Corpus

# Using params from Word2Vec_FastText_Comparison
params = {
    'alpha': 0.05,
    'size': 100,
    'window': 5,
    'iter': 5,
    'min_count': 5,
    'sample': 1e-4,
    'sg': 1,
    'hs': 0,
    'negative': 5,
}
model = Word2Vec(Text8Corpus(text8_path), **params)
wv = model.wv
print("Using trained model", wv)

Using trained model <gensim.models.keyedvectors.Word2VecKeyedVectors object at 0x7f8a8626bdf0>


In [6]:
!pip install annoy
!pip install gensim

In [7]:
from gensim.similarities.index import AnnoyIndexer

In [8]:

# 100 trees are being used in this example
annoy_index = AnnoyIndexer(model, 100)
# Derive the vector for the word "science" in our model
vector = wv["science"]
# The instance of AnnoyIndexer we just created is passed
approximate_neighbors = wv.most_similar([vector], topn=11, indexer=annoy_index)
# Neatly print the approximate_neighbors and their corresponding cosine similarity values
print("Approximate Neighbors")
for neighbor in approximate_neighbors:
    print(neighbor)

normal_neighbors = wv.most_similar([vector], topn=11)
print("\nExact Neighbors")
for neighbor in normal_neighbors:
    print(neighbor)

/Users/imayak/exper/python_nlp_venv/lib/python3.8/site-packages/gensim/similarities/index.py:198: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  index = AnnoyIndex(num_features)


Approximate Neighbors
('science', 1.0)
('actuarial', 0.6106608510017395)
('interdisciplinary', 0.6020962297916412)
('astrobiology', 0.598374605178833)
('scientific', 0.5858789682388306)
('psychohistory', 0.5811305642127991)
('multidisciplinary', 0.5804038345813751)
('sciences', 0.5768157243728638)
('humanities', 0.5753591656684875)
('interdisciplinarity', 0.5725245475769043)
('robotics', 0.569885641336441)

Exact Neighbors
('science', 0.9999999403953552)
('fiction', 0.7593916058540344)
('actuarial', 0.6968300342559814)
('interdisciplinary', 0.6833452582359314)
('astrobiology', 0.6773940920829773)
('xenobiology', 0.6714037656784058)
('technology', 0.6638299226760864)
('scientific', 0.6570075750350952)
('psychohistory', 0.6490969061851501)
('multidisciplinary', 0.6478780508041382)
('biostatistics', 0.6462464332580566)


In [14]:
fname = '/tmp/mymodel.index'

# Persist index to disk
annoy_index.save(fname)

# Load index back
import os.path
if os.path.exists(fname):
    annoy_index2 = AnnoyIndexer()
    annoy_index2.load(fname)
    annoy_index2.model = model

# Results should be identical to above
vector = wv["science"]
approximate_neighbors2 = wv.most_similar([vector], topn=11, indexer=annoy_index2)
for neighbor in approximate_neighbors2:
    print(neighbor)

assert approximate_neighbors == approximate_neighbors2

('science', 0.9998273665114539)
('actuarial', 0.6106608510017395)
('interdisciplinary', 0.6020962297916412)
('astrobiology', 0.598374605178833)
('scientific', 0.585878998041153)
('psychohistory', 0.5811306238174438)
('multidisciplinary', 0.5804038345813751)
('sciences', 0.5768157243728638)
('humanities', 0.5753591656684875)
('interdisciplinarity', 0.5725245475769043)
('robotics', 0.5698857009410858)


/Users/imayak/exper/python_nlp_venv/lib/python3.8/site-packages/gensim/similarities/index.py:168: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  self.index = AnnoyIndex(d['f'])


AssertionError: 